In [ ]:
import pandas as pd
import numpy as np

In [ ]:
persons = input("Enter the visiting guest names separating them with \",\": ")
persons = persons.lower().strip()
persons = persons.split(",")
persons = [i.strip() for i in persons]
print(persons)

In [ ]:
items = input("Enter the dishes separating them with \",\": ")
items = items.strip()
items = items.split(",")
items = [i.strip() for i in items]
print(items)

In [ ]:
costs = []
for i in items:
    costs.append(float(input("How much did {} cost? Answer: ".format(i))))
print(costs)
costs = dict(zip(items,costs))
print(costs)

In [ ]:
links = {}
for i in persons:
    links[i] = []
for i in persons:
    for j in items:
        temp = input("Did {} consume {}? Answer yes or no (y or n): ".format(i,j))
        if temp == "y" or temp == "yes":
            links[i].append(j)
            
print(links)

In [ ]:
dm = np.ndarray(shape = (len(items),len(persons)))
dm[0][1] = 1
print(dm)
for i in range(len(persons)):
    for j in range(len(items)):
        if items[j] in links[persons[i]]:
            dm[j][i] = 1
        else:
            dm[j][i] = 0
data = pd.DataFrame(data = dm, columns = persons)
data["sum"] = data.sum(axis = 1)
items = pd.Series(items)
data["items"] = items
data.head()

In [ ]:
costs = pd.DataFrame(costs,index = ["price"]).transpose()

costs.reset_index(inplace = True)
costs = costs.rename(columns = {'index':'items'})
costs.head()

In [ ]:
data = pd.merge(data, costs)
data.head()

In [ ]:
# column_sum
for i in persons:
    data[i+"_total"] = data[i]/data["sum"]*data["price"]

data = data.drop(persons + ["sum","items","price"],axis=1)
data.head()

In [ ]:
#Total Row
data.loc["Total_without_tip"] = data.sum()
data.head()

tip = float(input("What was the tip %? Write float without % sign: "))/100
data.loc["Total"] = data.loc["Total_without_tip"]*(1+tip)
data.head(10)

In [ ]:
# Mechanics of transactions
# 1. Pop the who pays guy and move to the end
who_pays = input("Who pays?: ").lower()
persons.remove(who_pays)
persons.append(who_pays)
print(persons)

# 2. Update totals
for i in range(len(persons)):
    if i > 0:
        data.loc["Total",persons[i]+"_total"] += data.loc["Total",persons[i-1]+"_total"]
print(data.head())

#3. Make Transactions
for i in range(len(persons)):
    if i == len(persons)-1:
        print("{} pays the bill".format(persons[i]))
    else:
        print("{} pays to {} {} tenge".format(persons[i],persons[i+1],data.loc["Total",persons[i]+"_total"]))